#  Neural Networks: Regression on House Pricing Dataset
We consider a reduced version of a dataset containing house sale prices for King County, which includes Seattle. It includes homes sold between May 2014 and May 2015.

https://www.kaggle.com/harlfoxem/housesalesprediction

For each house we know 18 house features (e.g., number of bedrooms, number of bathrooms, etc.) plus its price, that is what we would like to predict.

In [ ]:
#put here your ``numero di matricola''
numero_di_matricola = 1 # COMPLETE

## Insert your ID number ("numero di matricola") below

In [ ]:
#put here your ``numero di matricola''
numero_di_matricola = 1 # COMPLETE

In [ ]:
#import all packages needed
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Load the data, remove data samples/points with missing values (NaN) and take a look at them.

In [ ]:
#load the data
df = pd.read_csv('kc_house_data.csv', sep = ',')

#remove the data samples with missing values (NaN)
df = df.dropna()

df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,3.164000e+03,3.164000e+03,3164.000000,3164.000000,3164.000000,3.164000e+03,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000,3164.000000
mean,4.645240e+09,5.354358e+05,3.381163,2.071903,2070.027813,1.525054e+04,1.434893,0.009798,0.244311,3.459229,7.615676,1761.252212,308.775601,1967.489254,94.668774,98077.125158,47.557868,-122.212337,1982.544564,13176.302465
std,2.854203e+09,3.809004e+05,0.895472,0.768212,920.251879,4.254457e+04,0.507792,0.098513,0.776298,0.682592,1.166324,815.934864,458.977904,28.095275,424.439427,54.172937,0.140789,0.139577,686.256670,25413.180755
min,1.000102e+06,7.500000e+04,0.000000,0.000000,380.000000,6.490000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,380.000000,0.000000,1900.000000,0.000000,98001.000000,47.177500,-122.514000,620.000000,660.000000
25%,2.199775e+09,3.150000e+05,3.000000,1.500000,1430.000000,5.453750e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1950.000000,0.000000,98032.000000,47.459575,-122.324250,1480.000000,5429.500000
50%,4.027701e+09,4.450000e+05,3.000000,2.000000,1910.000000,8.000000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1545.000000,0.000000,1969.000000,0.000000,98059.000000,47.572500,-122.226000,1830.000000,7873.000000
75%,7.358175e+09,6.402500e+05,4.000000,2.500000,2500.000000,1.122250e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2150.000000,600.000000,1990.000000,0.000000,98117.000000,47.680250,-122.124000,2360.000000,10408.250000
max,9.839301e+09,5.350000e+06,8.000000,6.000000,8010.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,12.000000,6720.000000,2620.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,5790.000000,425581.000000


Extract input and output data. We want to predict the price by using features other than id as input.

In [ ]:
Data = df.values
# m = number of input samples
m = Data.shape[0]
print("Amount of data:",m)
Y = Data[:m,2]
X = Data[:m,3:]

Amount of data: 3164


## Data Pre-Processing

We split the data into 3 parts: one will be used for training and choosing the parameters, one for choosing among different models, and one for testing. The part for training and choosing the parameters will consist of $2/3$ of all samples, the one for choosing among different models will consist of $1/6$ of all samples, while the other part consists of the remaining $1/6$-th of all samples.

In [ ]:
# Split data into train (2/3 of samples), validation (1/6 of samples), and test data (the rest)
m_train = int(2./3.*m)
m_val = int((m-m_train)/2.)
m_test = m - m_train - m_val
print("Amount of data for training and deciding parameters:",m_train)
print("Amount of data for validation (choosing among different models):",m_val)
print("Amount of data for test:",m_test)
from sklearn.model_selection import train_test_split

Xtrain_and_val, Xtest, Ytrain_and_val, Ytest = train_test_split(X, Y, test_size=m_test/m, random_state=numero_di_matricola)
Xtrain, Xval, Ytrain, Yval = train_test_split(Xtrain_and_val, Ytrain_and_val, test_size=m_val/(m_train+m_val), random_state=numero_di_matricola)

Amount of data for training and deciding parameters: 2109
Amount of data for validation (choosing among different models): 527
Amount of data for test: 528


Let's standardize the data.

In [ ]:
# Data pre-processing
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(Xtrain)
Xtrain_scaled = scaler.transform(Xtrain)
Xtrain_and_val_scaled = scaler.transform(Xtrain_and_val)
Xval_scaled = scaler.transform(Xval)
Xtest_scaled = scaler.transform(Xtest)

## Neural Networks
Let's start by learning a simple neural network with 1 hidden node.
Note: we are going to use the input parameter solver='lbfgs' and random_state=numero_di_matricola to fix the random seed (so results are reproducible).

In [ ]:
#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(1,),solver="lbfgs",random_state=numero_di_matricola)

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
print(NN.intercepts_)

0.2639479253417457
0.30403685196514796
[array([[-213.32710074],
       [ 267.74308829],
       [ 520.9161202 ],
       [ -60.32534191],
       [   4.05927076],
       [ 707.00232121],
       [ 292.91931382],
       [ 135.7958694 ],
       [ 811.60932712],
       [ 490.45537523],
       [ 162.78453019],
       [-578.83804667],
       [  37.89891411],
       [-202.21239746],
       [ 596.83587404],
       [-141.33429569],
       [ 146.16707317],
       [ -26.73115152]]), array([[141.90559471]])]
[array([3773.45441811]), array([-26.37667981])]


## Neural Networks vs Linear Models

Let's learn a linear model on the other same data and compare the results with the simple NN above.

In [ ]:
#COMPLETE
from sklearn import linear_model

LR = linear_model.LinearRegression()

LR.fit(Xtrain_scaled,Ytrain)

print(1-LR.score(Xtrain_scaled,Ytrain))
print(1-LR.score(Xval_scaled,Yval))

print(LR.coef_)
print(LR.intercept_)

0.2653859924353308
0.31176613563743305
[-2.83881255e+04  3.46039033e+04  1.81896850e+18 -8.54378107e+03
 -6.05860287e+02  1.00433288e+05  4.20282698e+04  2.01823963e+04
  1.09874269e+05 -1.63501936e+18 -9.00806992e+17 -7.80771471e+04
  6.56033131e+03 -2.73989648e+04  8.35250951e+04 -2.13083018e+04
  2.27525614e+04 -2.25897712e+03]
536740.1865575869


Is there a way to make a NN network learn a linear model?

Let's first check what is the loss used by MLPRegressor...

In [ ]:
#let's write the code to learn a linear model with NN: how?

#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(1,),solver="lbfgs",random_state=numero_di_matricola,activation="identity")

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
print(NN.intercepts_)

0.26535942166590454
0.3115390658284922
[array([[  51.55070235],
       [ -59.02846704],
       [-122.96939596],
       [  13.19466306],
       [  -1.10694448],
       [-165.016906  ],
       [ -68.62491747],
       [ -32.12596414],
       [-183.34020234],
       [-114.96558392],
       [ -38.51551396],
       [ 128.83877577],
       [ -10.76237595],
       [  46.43749972],
       [-137.83874569],
       [  35.64844107],
       [ -36.32214078],
       [   3.29740834]]), array([[-607.24280969]])]
[array([-883.44724722]), array([365.29153421])]


Note that there is an $\ell_2$ regularization term in MLPRegressor. What about making it smaller?

In [ ]:
#COMPLETE

## More Complex NNs

Let's try more complex NN, for example increasing the number of nodes in the only hidden layer, or increasing the number of hidden layers.

Let's build a NN with 2 nodes in the only hidden layer

In [ ]:
#let's build a NN with 2 nodes in the only hidden layer

#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(2,),solver="lbfgs",random_state=numero_di_matricola)

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
print(NN.intercepts)

0.18062109906851975
0.2072188864064245
[array([[  91.0135717 ,  -33.31648469],
       [ 120.45601735,   39.31363437],
       [  85.74123108,   73.03537278],
       [-261.27351073,   28.30942803],
       [ -30.57377112,   17.91716977],
       [ 198.30951336,   25.30846506],
       [  34.87362728,   37.57703322],
       [  96.99039338,   25.69137306],
       [ 312.37605943,  132.89249257],
       [  85.00062032,   68.81506248],
       [  19.24883267,   23.38873098],
       [-217.42645617,  -81.21696339],
       [  -3.48529316,   20.11216403],
       [-301.2638864 ,  -26.39629252],
       [ 305.55023365,  144.59950929],
       [-463.83631129,  -16.406266  ],
       [ 194.59591574,   52.89555635],
       [-250.92315846,  -11.166943  ]]), array([[614.54319227],
       [548.81462018]])]


AttributeError: 'MLPRegressor' object has no attribute 'intercepts'

Let's build a NN with 5 nodes in the only hidden layer

In [ ]:
#let's build a NN with 5 nodes in the only hidden layer

#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(5,),solver="lbfgs",random_state=numero_di_matricola)

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
print(NN.intercepts)

Note that with a smaller number of iterations we had a larger error on training set but a smaller error on validation data -> "early stopping is a form of regularization"

Let's build a NN with 10 nodes in the only hidden layer

In [ ]:
#let's build a NN with 10 nodes in the only hidden layer

#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(10,),solver="lbfgs",random_state=numero_di_matricola)

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
#print(NN.intercepts)

0.12218155069751013
0.3077493637921128


C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's build a NN with 100 nodes in the only hidden layer. Note that this is the default!

In [ ]:
#let's build a NN with 100 nodes in the only hidden layer

#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(100,),solver="lbfgs",random_state=numero_di_matricola)

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
#print(NN.intercepts)

0.031062753470383675
0.48104846274722646


C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 1 node each

In [ ]:
#let's build a NN with 2 hidden layers, 1 node each

#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(1,1,),solver="lbfgs",random_state=numero_di_matricola)

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
#print(NN.intercepts)

0.24000826164233557
0.2730179156052638


C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 2 nodes each

In [ ]:
#let's build a NN with 2 layers, 2 nodes each

#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(2,2,),solver="lbfgs",random_state=numero_di_matricola)

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
#print(NN.intercepts)

0.21286091902027926
0.26866576815820586


C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 10 nodes each

In [ ]:
#let's build a NN with 2 layers, 10 nodes each

#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(10,10,),solver="lbfgs",random_state=numero_di_matricola)

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
#print(NN.intercepts)

0.07162884599270314
0.30154840167981534


C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Let's try 2 layers, 100 nodes each

In [ ]:
#let's build a NN with 2 layers, 100 nodes each

#let's load the MLPRegressor
from sklearn.neural_network import MLPRegressor

#let's define the model
#COMPLETE
NN = MLPRegressor(hidden_layer_sizes=(100,100,),solver="lbfgs",random_state=numero_di_matricola)

#let's learn the model on training data
#COMPLETE
NN.fit(Xtrain_scaled,Ytrain)

#let's print the error (1 - R^2) on training data
#COMPLETE
print(1-NN.score(Xtrain_scaled,Ytrain))

#let's print the error (1 - R^2) on validation data
#COMPLETE
print(1-NN.score(Xval_scaled,Yval))

#let's print the coefficients of the model for the input nodes (but not the bias)
#COMPLETE
#print(NN.coefs_)

#let's print the coefficient for the bias (i.e., the bias)
#COMPLETE
#print(NN.intercepts)

0.02300567407541343
0.33637502274767184


C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


So it seems that 1 layer (and default number of iterations) works best for this dataset. Let's try 5-fold cross-validation with number of nodes in the hidden layer between 1 and 20.
Note that we use train and validation data together, since we are doing cross-validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

param = {'hidden_layer_sizes':[(1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,),(12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,)]}
mlp = MLPRegressor(solver="lbfgs",random_state=numero_di_matricola)
#COMPLETE
GS = GridSearchCV(mlp,param,cv=5)

GS.fit(Xtrain_scaled,Ytrain)

C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERAT

GridSearchCV(cv=5, estimator=MLPRegressor(random_state=1, solver='lbfgs'),
             param_grid={'hidden_layer_sizes': [(1,), (2,), (3,), (4,), (5,),
                                                (6,), (7,), (8,), (9,), (10,),
                                                (11,), (12,), (13,), (14,),
                                                (15,), (16,), (17,), (18,),
                                                (19,), (20,)]})

Now let's check what is the best parameter, and compare the best NNs with the linear model (learned on train and validation) on test data.

In [ ]:
#let's print the best model according to grid search
#COMPLETE
print(GS.best_params_)

#let's print the error 1-R^2 for the best model
#COMPLETE
print(1-GS.best_score_)

{'hidden_layer_sizes': (19,)}
0.19298222268665965


Let compare the error of the best NN on train and validation and on test data.

In [ ]:
#COMPLETE
best_NN = MLPRegressor(**GS.best_params_,solver="lbfgs",random_state=numero_di_matricola)
best_NN.fit(Xtrain_scaled,Ytrain)

print("Training error: ", 1-best_NN.score(Xtrain_scaled,Ytrain))
print("Test error: ", 1-best_NN.score(Xtest_scaled,Ytest))
print("Validation error: ", 1-best_NN.score(Xval_scaled,Yval))

Training error:  0.0700557603625045
Test error:  0.19954939210569833
Validation error:  0.29968801028044023


C:\Users\USER\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Now let's learn the linear model on train and validation, and get error (1-R^2) on train and validation and on test data.

In [ ]:
#COMPLETE

Note: MLPRegressor has several other parameters!